In [0]:
%sql
select * from default.pubmed_filtrado_part where limit 3

pmid,title,pub_year,journal,volume,issn,language,country,citation,mid,doi,pii,pmc,fore,last,fullname,orcid,affiliation,affiliation_country,term,major,pub_year_clean
38774575,Antibacterial activity of Betadine (,2022,F1000Research,11,2046-1402,eng,England,"F1000Res, 11, 2022",,10.12688/f1000research.123777.2,,PMC11106593,Hendry,Rusdy,Hendry Rusdy,,"Department of Oral and Maxillofacial Surgery, Faculty of Dentistry Universitas Sumatera Utara, Medan, Sumatera Utara, 20155, Indonesia.",Indonesia,Pseudomonas aeruginosa,Y,2022
38662853,Agreement Between Physician Evaluation and the Composite Response Index in Diffuse Cutaneous Systemic Sclerosis.,2022,Arthritis care & research,74,2151-4658,eng,United States,"Arthritis Care Res (Hoboken), 74, 2022",,10.1002/acr.24638,,,Boyang,Zheng,Boyang Zheng,0000-0003-2509-5766,"McGill University, Jewish General Hospital, Montreal, Quebec, Canada.",Canada,Adult,N,2022
38774575,Antibacterial activity of Betadine (,2022,F1000Research,11,2046-1402,eng,England,"F1000Res, 11, 2022",,10.12688/f1000research.123777.2,,PMC11106593,Hendry,Rusdy,Hendry Rusdy,,"Department of Oral and Maxillofacial Surgery, Faculty of Dentistry Universitas Sumatera Utara, Medan, Sumatera Utara, 20155, Indonesia.",Indonesia,Plant Extracts,Y,2022


In [0]:
from pyspark.sql.functions import col, trim, lower, substring

# Carga tabla original
df = spark.table("default.pubmed_filtrado_part")

# Normaliza columnas
df_clean = df.select(
    substring("pub_year_clean", 1, 4).alias("pub_year_for"),
    "pmid",
    lower(trim(col("fullname"))).alias("fullname"),
    lower(trim(col("orcid"))).alias("orcid"),
    lower(trim(col("fore"))).alias("fore"),
    lower(trim(col("last"))).alias("last"),
    lower(trim(col("affiliation"))).alias("affiliation"),
    lower(trim(col("affiliation_country"))).alias("affiliation_country"),
).dropDuplicates()

# Guarda como tabla particionada
df_clean.write \
    .mode("overwrite") \
    .partitionBy("pub_year_for") \
    .format("parquet") \
    .save("dbfs:/FileStore/pubmed_tables_parquet/pubmed_autores/")

In [0]:
%sql
USE CATALOG hive_metastore;

CREATE TABLE IF NOT EXISTS default.pubmed_autores
  USING PARQUET
  LOCATION 'dbfs:/FileStore/pubmed_tables_parquet/pubmed_autores/';

-- repara las particiones
 MSCK REPAIR TABLE default.pubmed_autores; 

In [0]:

dt_tmp =spark.sql("""
    SELECT 
    pmid,
    fullname,
    affiliation,
    COUNT(DISTINCT orcid) AS num_orcid_distintos,
    SUM(CASE WHEN orcid IS NOT NULL THEN 1 ELSE 0 END) AS tiene_orcid
    FROM 
    default.pubmed_autores
    GROUP BY 
    pmid, fullname, affiliation
    HAVING 
    num_orcid_distintos > 1
""")

dt_tmp.createOrReplaceTempView("v_autor_duplicado")

spark.sql("""
     CREATE TABLE IF NOT EXISTS default.pubmed_autores_clean
   -- USING DELTA
    AS SELECT * FROM default.pubmed_autores
    where not exists (select 1 from v_autor_duplicado v where default.pubmed_autores.pmid = v.pmid and default.pubmed_autores.fullname = v.fullname and default.pubmed_autores.affiliation = v.affiliation)
    union all
    SELECT * FROM default.pubmed_autores
    where  exists (select 1 from v_autor_duplicado v where default.pubmed_autores.pmid = v.pmid and default.pubmed_autores.fullname = v.fullname and default.pubmed_autores.affiliation = v.affiliation  )
    and default.pubmed_autores.orcid is not null

   """)






DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# 📌 1️⃣ Carga tabla original
from pyspark.sql.functions import col, regexp_replace, when, length, trim

df = spark.table("default.pubmed_autores_clean")

# 📌 2️⃣ Normaliza: quita texto extra y deja solo números

df_normali = df.withColumn(
    "orcid_normalized",
    when(
        col("orcid").isNotNull(),
        trim(regexp_replace(
            regexp_replace(trim(col("orcid")), r"https?://orcid\.org/", ""),
            "-", ""
        ))
    ).otherwise(None)
)


# 📌 3️⃣ Inserta guiones si hay 16 dígitos
from pyspark.sql.functions import col, when, length, trim, concat, lit

df_final = df_normali.withColumn(
    "orcid_normalized2",
    when(
        length(trim(col("orcid_normalized"))) == 16,
        concat(
            trim(col("orcid_normalized")).substr(1, 4), lit("-"),
            trim(col("orcid_normalized")).substr(5, 4), lit("-"),
            trim(col("orcid_normalized")).substr(9, 4), lit("-"),
            trim(col("orcid_normalized")).substr(13, 4)
        )
    ).otherwise(
        when(
            length(trim(col("orcid_normalized"))) < 16,
            None
        ).otherwise(trim(col("orcid_normalized")))
    )
)

df_final.select("orcid", "orcid_normalized","orcid_normalized2").show(100, truncate=False)


# Registros con ORCID válido
df_with_orcid = df_final.filter(
    (col("orcid_normalized2").isNotNull()) & (trim(col("orcid_normalized2")) != "")
)

#📌 1️⃣ Filtra registros sin ORCID válido
df_without_orcid = df_final.filter(
   (col("orcid_normalized2").isNull()) | (trim(col("orcid_normalized2")) == "")
)

# Join para intentar recuperar ORCID por coincidencia exacta
df_recovered = df_without_orcid.alias("a").join(
    df_with_orcid.alias("b"),
    (col("a.fullname") == col("b.fullname")) &
    (col("a.affiliation") == col("b.affiliation")),
    how="left"
).select(
    col("a.*"),
    col("b.orcid_normalized2").alias("orcid_recovered")
)

##print ( df_recovered.filter(
##   (col("orcid_recovered").isNotNull()) 
##).count()
##)



+-------------------------------------+----------------+-------------------+
|orcid                                |orcid_normalized|orcid_normalized2  |
+-------------------------------------+----------------+-------------------+
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |
|                                     |                |NULL               |

In [0]:
# ✅ Combinar tabla original con ORCID recuperados
# -------------------------------------------------
# 1️⃣ Importa librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, lower, regexp_replace, coalesce

# 2️⃣ Inicia sesión Spark (en Databricks ya viene configurado)
spark = SparkSession.builder.getOrCreate()

# 3️⃣ Carga tus DataFrames
##df_final = spark.table("mi_db.mi_tabla_autores_final")
###df_recovered = spark.table("mi_db.mi_tabla_autores_recovered")

# 4️⃣ Normaliza claves (opcional, pero recomendable)
df_final_clean = df_final.withColumn(
    "fullname_norm",
    lower(trim(regexp_replace(col("fullname"), "[^a-zA-Z0-9 ]", "")))
).withColumn(
    "affiliation_norm",
    lower(trim(regexp_replace(col("affiliation"), "[^a-zA-Z0-9 ]", "")))
)

df_recovered_clean = df_recovered.withColumn(
    "fullname_norm",
    lower(trim(regexp_replace(col("fullname"), "[^a-zA-Z0-9 ]", "")))
).withColumn(
    "affiliation_norm",
    lower(trim(regexp_replace(col("affiliation"), "[^a-zA-Z0-9 ]", "")))
)

# 5️⃣ JOIN por claves normalizadas
df_merged = df_final_clean.alias("f").join(
    df_recovered_clean.alias("r"),
    on=[
        col("f.pmid") == col("r.pmid"),
        col("f.fullname_norm") == col("r.fullname_norm"),
        col("f.affiliation_norm") == col("r.affiliation_norm")
    ],
    how="left"
)

# 6️⃣ Combina ORCID original o recuperado
df_merged = df_merged.withColumn(
    "orcid_final",
    coalesce(col("f.orcid_normalized2"), col("r.orcid_recovered"))
)

# 7️⃣ Muestra resultado (opcional)
df_merged.select(
    col("f.pmid").alias("pmid"),
    col("f.fullname").alias("fullname"),
    col("f.affiliation").alias("affiliation"),
    col("f.orcid_normalized2").alias("orcid_original"),
    col("r.orcid_recovered").alias("orcid_recuperado"),
    col("orcid_final")
).show(20, truncate=False)

# 8️⃣ Guarda tabla final en formato Delta
#df_merged.write.mode("overwrite").format("delta").saveAsTable("mi_db.mi_tabla_autores_orcid_merged")

print("✅ Tabla final con ORCID combinados guardada: mi_db.mi_tabla_autores_orcid_merged")


+--------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------+-------------------+
|pmid    |fullname           |affiliation                                                                                                                                                        |orcid_original     |orcid_recuperado   |orcid_final        |
+--------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------+-------------------+
|23401628|aydın akyuz        |department of cardiology, faculty of medicine, namık kemal university medical school, turkey.                                                                      |NULL               |NULL               |N

In [0]:
df_merged.filter(
    (col("f.orcid_normalized2").isNull()) & (trim(col("orcid_final")).isNotNull())
).show(3)


+--------+--------------------+-----+--------------+--------------+--------------------+------------+----------------+-----------------+--------------------+--------------------+--------+--------------------+-----+--------------+--------------+--------------------+------------+----------------+-----------------+-------------------+--------------------+--------------------+-------------------+
|    pmid|            fullname|orcid|          fore|          last|         affiliation|pub_year_for|orcid_normalized|orcid_normalized2|       fullname_norm|    affiliation_norm|    pmid|            fullname|orcid|          fore|          last|         affiliation|pub_year_for|orcid_normalized|orcid_normalized2|    orcid_recovered|       fullname_norm|    affiliation_norm|        orcid_final|
+--------+--------------------+-----+--------------+--------------+--------------------+------------+----------------+-----------------+--------------------+--------------------+--------+--------------------+

In [0]:



from pyspark.sql.functions import max

# 1️⃣ Lista de campos para agrupar: todos excepto orcid y orcid_final
group_cols = [
    "f.pub_year_for",
    "f.pmid",
    "f.fullname",
    "f.fore",
    "f.last",
    "f.affiliation"
]

# 2️⃣ Agrupar y devolver el máximo ORCID
df_to_save = df_merged.groupBy(group_cols).agg(
    max("orcid_final").alias("orcid_final")
)

df_to_save.write.mode("overwrite").format("delta").saveAsTable("default.pubmed_art_autor_aff")

In [0]:
%sql


select count(1) from default.pubmed_art_autor_aff

count(1)
38401193


In [0]:
%sql
--  7321048

-- select 2762074 - 2087258 from dual
--9 665 880
--select count(1) 
--from 
--(
select    pmid,
          fullname,
          affiliation, 
          count(1)
           from default.pubmed_art_autor_aff
group by pmid,
          fullname,
          affiliation
having count(1)         > 1
--)



 --select * from unirdat.pubmed_db.m_articulo_autor
 --where pmid = 33640246	and fullname ='bagher larijani'


pmid,fullname,affiliation,count(1)
34255852,dineke zeegers paget,eupha executive director.,2
33462189,a jeremy willsey,"institute for neurodegenerative diseases, ucsf weill institute for neurosciences, university of california san francisco, san francisco, ca, usa.",2
34473588,dea schröder-van den nieuwendijk,"department of dietetics, roessingh rehabilitation centre, enschede, the netherlands.",2
32492397,anh ngoc thi nguyen,"department of genomics and cytogenetics, institute of biomedicine and pharmacy (ibp), vietnam military medical university, vietnam.",2
35126748,oleg vasylovich gerush,"pharmacy department, bukovinian state medical university, chernivtsi, ukraine.",2


In [0]:
dt_tmp =spark.sql("""
  select    pmid,
          fullname,
          affiliation, 
          count(1) as total
           from default.pubmed_art_autor_aff
group by pmid,
          fullname,
          affiliation
having total        > 1
""")

dt_tmp.createOrReplaceTempView("v_autor_duplicado2")

spark.sql("""
  delete from default.pubmed_art_autor_aff
  where  exists (select 1 from v_autor_duplicado2 v where default.pubmed_art_autor_aff.pmid = v.pmid and default.pubmed_art_autor_aff.fullname = v.fullname and default.pubmed_art_autor_aff.affiliation = v.affiliation)
   
   """)



DataFrame[num_affected_rows: bigint]

In [0]:
%sql
SELECT count(1)
FROM default.pubmed_art_autor_aff a


count(1)
38401193


In [0]:
%sql
SELECT count(1)
FROM default.pubmed_art_autor_aff a
where  orcid_final is not null

--28769836

count(1)
12761846


In [0]:
import requests
import pandas as pd
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# -----------------------------------------------------------------------------
# 1️⃣ Spark session
# -----------------------------------------------------------------------------
#spark = SparkSession.builder.getOrCreate()

# -----------------------------------------------------------------------------
# 2️⃣ Config
# -----------------------------------------------------------------------------
API_RATE_LIMIT = 1
MAX_RECORDS =10000
BATCH_SIZE = 100

# -----------------------------------------------------------------------------
# 3️⃣ Leer tabla
# -----------------------------------------------------------------------------

query = """
SELECT a.*
FROM default.pubmed_art_autor_aff a
where  orcid_final is null
limit 5000
"""





#df_autores_all = spark.table("unirdat.pubmed_db.m_articulo_autor")
df_autores_all = spark.sql(query)
df_autores_all.show(3)
df_autores = df_autores_all.select ( "pmid", "fullname", "affiliation" ).distinct()  
df_sin_orcid = df_autores.filter(col("orcid_final").isNull())

df_sin_orcid_pd = df_sin_orcid.limit(MAX_RECORDS).select(
    "pmid", "fullname", "affiliation"
).toPandas()

total = len(df_sin_orcid_pd)

# -----------------------------------------------------------------------------
# 4️⃣ Función API
# -----------------------------------------------------------------------------
def buscar_orcid(fullname, affiliation):
    base_url = "https://pub.orcid.org/v3.0/search/?q="
    query = f'given-names:{fullname}'
    headers = {"Accept": "application/json"}

    response = requests.get(base_url + query, headers=headers, timeout=10)
    if response.status_code == 200:
        data = response.json()
        items = data.get("result", [])
        if items:
            return items[0]['orcid-identifier']['path']
    return None

# -----------------------------------------------------------------------------
# 5️⃣ Loop por lotes + guardado incremental
# -----------------------------------------------------------------------------
for i in range(0, total, BATCH_SIZE):
    batch = df_sin_orcid_pd.iloc[i:i+BATCH_SIZE]
    resultados = []

    print(f"🔍 Procesando registros {i+1} a {i+len(batch)} de {total}")

    for _, row in batch.iterrows():
        orcid_id = buscar_orcid(row["fullname"], row["affiliation"])
        resultados.append({
            "pmid": row["pmid"],
            "fullname": row["fullname"],
            "affiliation": row["affiliation"],
            "orcid_api": orcid_id
        })
        time.sleep(API_RATE_LIMIT)

    # ⚡ Cada lote se convierte a Spark y se inserta como append
    df_resultados = pd.DataFrame(resultados)
    df_resultados_spark = spark.createDataFrame(df_resultados)

    df_resultados_spark.write.mode("append").format("delta").saveAsTable(
        "unirdat.pubmed_db.m_articulo_autor_orcid_api"
    )

    print(f"✅ Lote {i//BATCH_SIZE + 1} guardado")




+------------+--------+--------------------+--------+------------------+--------------------+-----------+
|pub_year_for|    pmid|            fullname|    fore|              last|         affiliation|orcid_final|
+------------+--------+--------------------+--------+------------------+--------------------+-----------+
|        2020|30704332|mipattra kwanmong...|mipattra|  kwanmongkolthong|school of physica...|       NULL|
|        2020|30973764|heleen a reinders...|heleen a|reinders-messelink|rehabilitation ce...|       NULL|
|        2020|30975622|       mukund thatte|  mukund|            thatte|department of pla...|       NULL|
+------------+--------+--------------------+--------+------------------+--------------------+-----------+
only showing top 3 rows
🔍 Procesando registros 1 a 100 de 500
✅ Lote 1 guardado
🔍 Procesando registros 101 a 200 de 500
✅ Lote 2 guardado
🔍 Procesando registros 201 a 300 de 500
✅ Lote 3 guardado
🔍 Procesando registros 301 a 400 de 500
✅ Lote 4 guardado
🔍 Pr

In [0]:

#spark.sql("RESTORE TABLE unirdat.pubmed_db.m_articulo_autor_orcid_api TO VERSION AS OF 7")



DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [0]:
%sql
SHOW TABLES IN unirdat.pubmed_db;


database,tableName,isTemporary
pubmed_db,m_articulo,false
pubmed_db,m_articulo_autor,false
pubmed_db,m_articulo_autor_orcid_api,false
pubmed_db,m_autor_afiliacion_ror_normalized,false
pubmed_db,n_affiliaciones,false
pubmed_db,n_articulo,false
pubmed_db,n_language,false
pubmed_db,n_orcdi,false
pubmed_db,n_pais,false
,_sqldf,true


In [0]:
%sql
--28769221
update default.pubmed_art_autor_aff  a
set  orcid_final= (select MAX(orcid_api) from unirdat.pubmed_db.m_articulo_autor_orcid_api b
                       where 
                        a.pmid = b.pmid
                    AND a.fullname = b.fullname
                    AND a.affiliation = b.affiliation
                       )
where orcid_final is null
 

num_affected_rows
28776449


In [0]:
%sql

select count(1) from default.pubmed_art_autor_aff
where orcid_final is not null
--select count(1) from (
--select distinct fullname, affiliation from unirdat.pubmed_db.m_articulo_autor
--where orcid_final is null
--)
--truncate table pubmed_db.m_articulo_autor_orcid_api
--select * from unirdat.pubmed_db.m_articulo_autor_orcid_api

count(1)
9632471


In [0]:
%sql
--select fullname, count(1) from unirdat.pubmed_db.m_articulo_autor_orcid_api
--group by fullname
--having count(1) > 1;

select * from unirdat.pubmed_db.m_articulo_autor_orcid_api
where fullname = 'jingjing xiang'

pmid,fullname,affiliation,orcid_api
31304847,jingjing xiang,"centre for reproduction and genetics, suzhou municipal hospital, suzhou, jiangsu province, china.",0009-0003-7622-3382
31304847,jingjing xiang,"centre for reproduction and genetics, the affiliated suzhou hospital of nanjing medical university, suzhou, jiangsu province, china.",0009-0003-7622-3382


In [0]:
%sql
drop table unirdat.pubmed_db.m_autor

In [0]:

# adjuntar un valor 
from pyspark.sql.functions import concat_ws, lower, trim, sha2, col, when

# 1️⃣ Normaliza y concatena los campos clave
df_autor =spark.table("default.pubmed_art_autor_aff")
df_autor_id = df_autor.withColumn(
    "autor_concat",
    concat_ws(
        "|",
        lower(trim(col("fullname"))),
        lower(trim(when(col("affiliation").isNotNull(), col("affiliation")).otherwise("")))
        
    )
)

# 2️⃣ Genera el hash SHA-256
df_con_hash = df_autor_id .withColumn(
    "author_id",
    sha2(col("autor_concat"), 256)
)

# 3️⃣ Prioriza ORCID si existe, si no usa el hash
df_final = df_con_hash.withColumn(
    "author_id",
    when(
        col("orcid_final").isNotNull() & (trim(col("orcid_final")) != ""),
        col("orcid_final")
    ).otherwise(col("author_id"))
)
#.drop("autor_concat", "author_id_sintetico")

df_final.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("default.pubmed_art_autor_aff")

df_autor = df_final.select(
    "author_id",
    "fullname",
    "fore",
    "last",
    col("orcid_final").alias("orcid"),

).distinct()


df_autor.write.mode("overwrite").format("delta").saveAsTable("unirdat.pubmed_db.m_autor")

df_final.show(truncate=False)

+------------+--------+----------------------------------------+-------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|pub_year_for|pmid    |fullname                                |fore         |last                      |affiliation                                                                                                                                     |orcid_final        |autor_concat                                                                                                                                                    |author_id          |
+------------+--------+----------------------------------------+-------------+------------------

In [0]:
%sql
--select *
--from 
--( 
   select count(1) from  default.pubmed_art_autor_aff
   where orcid_final is not null  and author_id <> orcid_final
-----)
--where t < 25

count(1)
0


In [0]:
from pyspark.sql.functions import max

df_autor_unico = df_autor.groupBy("author_id").agg(
    max("fullname").alias("fullname"),
    max("fore").alias("fore"),
    max("last").alias("last"),
    max("orcid").alias("orcid")
)


df_autor_unico.write.mode("overwrite").format("delta").saveAsTable("unirdat.pubmed_db.m_autor")

df_final.show(truncate=False)

+------------+--------+----------------------------------------+-------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|pub_year_for|pmid    |fullname                                |fore         |last                      |affiliation                                                                                                                                     |orcid_final        |autor_concat                                                                                                                                                    |author_id          |
+------------+--------+----------------------------------------+-------------+------------------

In [0]:
%sql
select orcid, count(1) from unirdat.pubmed_db.m_autor
where orcid is not null
group by orcid
having count(1) > 1

orcid,count(1)


In [0]:
%sql
--25255185
--   2091988
select * from unirdat.pubmed_db.m_articulo_autor
where fullname= 'bin zhang' and orcid_final is not null
-- 109



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can